<a href="https://colab.research.google.com/github/sanisa50/recipes/blob/main/Recipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" Cloning git repository to colab for resource file(s) """
!git clone https://github.com/sanisa50/recipes-etl.git
""" Installing and importing necessary packages """
!pip install fuzzywuzzy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


""" Reading json file from path into a Dataframe """
df = pd.read_json("/content/recipes-etl/recipes.json", lines=True)

""" Checking a few values and columns """
df.head()

""" Preprocessing the string by converting all words under the 'ingredients' column to lowercase. It's easier for comparing.
split() as the name name suggests splits each element of the 'ingredients' column (separated by space by default) and stores them into a list """
df["ingredient_lists"] = df["ingredients"].str.lower().str.split()


""" The fuzzywuzzy package has a module called process that allows you to calculate the string with the highest similarity out of a vector of strings.

Creating a function which accepts 3 arguments
1. A list of ingredients to compare against
2. A string to match against a the above list (Chilies in our case)
3. Threshold to assign a boolean value based on the similarity score given by process.extract() """

def doesMatch(ingredientsList, str2Match, threshold):

  """selecting the string with the highest matching percentage """
  maxFuzzyMatchScoreTuple = process.extractOne(str2Match, ingredientsList, scorer=fuzz.ratio)

  if maxFuzzyMatchScoreTuple[1] >= threshold:
    return maxFuzzyMatchScoreTuple, True
  else: 
    return maxFuzzyMatchScoreTuple, False

""" Calling the function doesMatch() and providing the String to match and threshold and mapping to all values in column ingredient_lists """
df['does_match'] = list(map(lambda ingredientsList: doesMatch(ingredientsList, 'Chilies', 83), df['ingredient_lists']))

""" Selecting all rows where above condition fulfills and dropping extra columns that we created above. """
df = df.loc[[i[1] for i in df['does_match']]].drop(['ingredient_lists', 'does_match'], axis=1)



""" Creating a function reformat() which extracts the numerical part and dimensional part of the cookTime and PrepTime column and converts the numerical part to minutes.
It accepts a column name which we want to perform above mentioned operations on """
def reformat(col):
  """ Removing the word 'PT' from values of the column passed """
  condition = df[col].str.contains("PT")
  df[col + "_v2"] = df.loc[condition, col].str.replace('PT', '')

  """ Extracting the numercial part """
  df[col + "value"] = df[col + "_v2"].str.extract('([0-9]+)')
  """ Converting it into number """
  df[col + "value"] = pd.to_numeric(df[col + "value"])
  df[col + "dimension"] = df[col + "_v2"].str.replace('[0-9]+', '')

  """ Converting all the values to minutes """
  df.loc[df[col + "dimension"] == "H", col + "Mins"] = df[col + "value"]*60
  df.loc[df[col + "dimension"] == "HM", col + "Mins"] = df[col + "value"]*60
  df.loc[df[col + "dimension"] == "M",  col + "Mins"] = df[col + "value"]
  df.loc[df[col + "dimension"] == "", col + "Mins"] = 0

""" Calling the function """
reformat('cookTime')
reformat('prepTime')

""" Adding the cooking time and prep time """
df['totalTime'] = df['prepTimeMins'] + df['cookTimeMins']
""" df['totalTime'].isnull() """
df['totalTime'] = df['totalTime'].fillna(0)


""" create a list of conditions """
conditions = [
    (df['totalTime'] > 60),
    (df['totalTime'] >= 30) & (df['totalTime'] <= 60),
    (df['totalTime'] < 30) ,
    (df['totalTime'] == 0)
    ]

""" create a list of the values we want to assign for each condition """
values = ['Hard', 'Medium', 'Easy', 'Unknown']

""" create a new column and use np.select to assign values to it using our lists as arguments """
df['difficultyLevel'] = np.select(conditions, values)

""" display updated DataFrame """
df.head()

""" Dropping extra columns """
df.drop(['cookTime_v2', 'cookTimevalue', 'cookTimedimension', 'cookTimeMins', 'prepTime_v2', 'prepTimevalue', 'prepTimedimension', 'prepTimeMins', 'totalTime'], axis=1).to_csv('assortedRecipes.csv', index = False)